In [1]:
from case.jobcrawler import webrequests
# from case.jobcrawler import webrequests
# from case.jobcrawler import Storage
# from case.jobcrawler import storagebase

ModuleNotFoundError: No module named 'case'

In [42]:
import case
job = case.jobcrawler.jobobj.Job()

AttributeError: module 'case.jobcrawler' has no attribute 'jobobj'

In [41]:
import case.demo

In [1]:
# %load jobcrawler.py
'''
需求：
针对职友集岗位数据爬取
1、城市数据通过列表修改
2、岗位数据通过列表修改
3、爬取1、2中所有城市对应所有岗位数据，存储到jobs.txt
'''
import time
import random
from bs4 import BeautifulSoup
from case.jobcrawler import webrequests
from case.jobcrawler import jobobj
from case.jobcrawler import storagebase


# 岗位数据爬取类
class JobCrawler:

    # 加载配置文件
    def __load_conf(self):
        citys = []
        jobs = []
        with open('./config/city.conf', encoding='utf-8') as f:
            for city in f:
                citys.append(city.strip())

        with open('./config/jobname.conf', encoding='utf-8') as f:
            for job in f:
                jobs.append(job.strip())

        return citys, jobs

    # 通过城市、岗位、页数定位网页
    def start(self):

        citys, jobs = self.__load_conf()

        for city in citys:
            for job in jobs:
                page = 1
                while True:
                    url = f'https://www.jobui.com/jobs?jobKw={job}&cityKw={city}&n={page}'
                    print(f"开始爬取{city}岗位为{job}的第{page}页数据")
                    flag = self.parser_bs(url, city)
                    if flag:
                        page += 1
                        time.sleep(random.randint(1, 2))
                    else:
                        break

    # 针对岗位基本信息进行爬取及解析
    def parser_bs(self, url, city):
        # # 【2】获取URL文本内容
        data = webrequests.get_data(url)
        soup = BeautifulSoup(data, 'html.parser')
        job_lists = soup.find_all('div', attrs={'class': 'c-job-list'})
        if job_lists:
            # 获取基本信息
            self.parser_info(job_lists, city)
            return True
        else:
            return False

    # 针对网页数据进行解析
    def parser_info(self, job_lists, city):
        for item in job_lists:
            job = jobobj.Job()
            job.jobname = item.find('h3').text
            spans = item.find('div', attrs={'class': 'job-desc'}).find_all('span')
            job.exp = spans[0].text
            job.degree = spans[1].text
            job.salary = spans[2].text
            job.company = item.find('a', attrs={'class': 'job-company-name'}).text
            # 点击量信息 coding...
            job.hit = item.find('span', attrs={'class': 'job-desc'}).text.strip()

            # 岗位详情地址
            jobaddr = item.find('a', attrs={'class': 'job-name'}).get('href')
            joburl = f'https://www.jobui.com/{jobaddr}'

            # time.sleep(random.randint(1, 5))
            # 获取详情信息
            self.parser_detail(joburl, job)
            # 数据库存储
            storage = storagebase.JobStorage()
            storage.insert(job)

    # 针对岗位详情页进行数据获取及解析
    def parser_detail(self, url, job):

        try:
            data = webrequests.get_data(url)
            soup = BeautifulSoup(data, 'html.parser')
            # 更新时间
            job.updatetime = soup.find('span', attrs={'class': 'fs16 gray9'}).text.strip()
            # 岗位详情、地址、来源网站  coding.... ------待完成
            
            pass
        except Exception as e:
            print(e)




if __name__ == '__main__':
    jobCrawler = JobCrawler()
    jobCrawler.start()


开始爬取北京岗位为elt的第1页数据


NameError: name 'webrequests' is not defined

In [4]:
class:

    def __init__(self):
        # 定义Job类中的成员属性
        self.jobname = ''
        self.exp = ''
        self.degree = ''
        self.salary = 10000
        self.company = ''
        self.hit = 1000
        self.industry = ''
        self.scale =''
        self.city = '北京'

        #岗位职责
        self.duty = ''
        #更新时间
        self.updatetime = ''
        #位置信息
        self.location = ''
        #数据获取时间
        self.collecttime = ''
        #网站来源
        self.website= ''

In [6]:
import requests

# 获取网页数据文本信息
def get_data(url, encoding='utf8'):
    headers = {'User-Agent': get_headers()}
    response = requests.get(url, headers=headers)
    response.encoding = encoding
    data = response.text
    return data


# 获取headers
def get_headers():
    return 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'


def parser_detail(url, job):

        try:
            data = get_data(url)
            soup = BeautifulSoup(data, 'html.parser')
            # 更新时间
            job.updatetime = soup.find('span', attrs={'class': 'fs16 gray9'}).text.strip()
            # 岗位详情、地址、来源网站  coding.... ------待完成
            job.jobname = soup.find('span', attrs={'class': 'pos_name'}).text.strip()
            job.exp = soup.find('span', attrs={'class': 'pos_require'}).text.strip()
            job.degree = soup.find('span', attrs={'class': 'pos_require'}).text.strip()
            job.salary = soup.find('span', attrs={'class': 'pos_salary'}).text.strip()
            job.company = soup.find('span', attrs={'class': 'pos_info'}).text.strip()
            job.website= soup.find('span', attrs={'class': 'report'}).text.strip()
            pass
        except Exception as e:
            print(e)
            
            
url = 'http://www.jobcn.com/position/detail.xhtml?redirect=0&posId=4202327&comId=465792&s=search/advanced&acType=1'
job = Job()
parser_detail(url, job)
            

'NoneType' object has no attribute 'text'
